Relatório do trabalho
==============
**Nome:** Álvaro Leandro Cavalcante Carneiro
**Linguagem utilizada:** Python 3.6

Os códigos e o relatório foram desenvolvidos em um Jupyter notebook e sua versão online pode ser encontrada aqui: https://colab.research.google.com/drive/1HQIBTnHCrcSrfAUOsx7836wZzDnQ9Fhd?usp=sharing. Nessa versão é possível executar novamente os algoritmos.

# Qual o problema ? 
Utilizar uma rede neural no modelo perceptron para identificar 3 classes de flores baseados em suas características de tamanho de pétalada e sépala.

## Importação das bibliotecas
Ferramentas usadas no processo de desenvolvimento

In [4]:
import random
import numpy as np
import pandas as pd
import math

## Análise Exploratória dos Dados (AED)
Vamos entender um pouco mais do nosso dataset, e deixar os dados em um formato que seja favorável para nossa rede neural perceptron.

O primeiro passo foi ler o arquivo baixado aqui: http://archive.ics.uci.edu/ml/datasets/Iris.
O arquivo em questão não possuia colunas, portanto eu modifiquei o mesmo para adicionar o nome das colunas na primeira linha e deixar o arquivo no formato .CSV ao invés do .DATA. 

In [5]:
dataframe = pd.read_csv('/home/alvaro/Documentos/mestrado/computação bio/redes neurais/datasets/iris2.csv', header = 0)

dataframe.head()

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Um passo inicial importante é verificar se o nosso dataset possui inconsistências quanto aos valores, podendo ser algum outlier, ruído, valor vazio, etc...

In [6]:
print('Valores nulos:')
print(dataframe.isna().sum())
dataframe.describe()

Valores nulos:
sepal-length    0
sepal-width     0
petal-length    0
petal-width     0
class           0
dtype: int64


,sepal-length,sepal-width,petal-length,petal-width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


O método *isnul()* nos mostrou que não há nenhum registro vazio no nosso dataset, além disso, é possível observar que os valores também parecem estar todos coerentes, sem a presença de outliers, como podemos notar pelo desvio padrão e mínimo e máximo de cada coluna.

Todavia, existe uma variação relativamente grande dentro do nosso domínio de atributos previsores. O atributo *petal-width* por exemplo, tem uma média de valor de 1.1, enquanto o *sepal-length* possui uma média de 5.8, portanto se faz necessário a padronização dos valores, para que nosso ajuste dos pesos não seja muito influenciado por essa diferença no tamanho da entrada. 

O tipo de normalização escolhido foi o **Z-score**, de forma arbitrária, por ser bastante comum em problemas como esse. Sua fórmula é bastante simples e foi representada no método *normalizacao_z_score*.

Ainda um pouco antes de fazer a normalização, dividi nosso *dataframe* em uma variável chamada **previsores** e outra chamada **classe**, como o nome sugere, os previsores são as colunas com as características das flores que serão usadas para tentar ajustar nossos pesos da rede de maneira à generalizar uma solução que encontre as classes corretamente.

In [10]:
previsores = dataframe.iloc[:, 0:4] 
classe = dataframe['class']

In [11]:
def normalizacao_z_score(valor):
    media = previsores[valor.name].mean()
    desvio_padrao = previsores[valor.name].std()

    return (valor - media) / desvio_padrao

O método *apply()* com a **lambda** aplicam o processo matemático do nosso método de normalização em cada um dos registros do *dataframe*.

In [15]:
previsores = previsores.apply(lambda row: normalizacao_z_score(row) )
previsores.head()

,sepal-length,sepal-width,petal-length,petal-width
0,-0.897674,1.028611,-1.336794,-1.308593
1,-1.139200,-0.124540,-1.336794,-1.308593
2,-1.380727,0.336720,-1.393470,-1.308593
3,-1.501490,0.106090,-1.280118,-1.308593
4,-1.018437,1.259242,-1.336794,-1.308593


Podemos ver agora que nossos valores estão em uma escala completamente diferente, todavia isso os torna mais coerente de serem trabalhados.

## Implementação da função de soma
O perceptron, assim como o perceptron multicamadas, possui diversas operações que são realizadas entre os neurônios, esses operações foram representadas em diferentes métodos peqenos.
O primeiro deles é a função de soma, que acontece em todos os neurônios, somando o valor do produto da multiplicação entre o neurônio adjacente anterior com o peso da sinapse artificial.
Isso foi feito utilizando a biblioteca do numpy por uma questão de performance, onde a função dot nos retorna o produto escalar que desejamos.

In [ ]:
def somatoria(entradas, pesos):
    return np.dot(entradas, pesos) 

## Função de ativação
A função de ativação por default no perceptron é a chamada "step function (função degrau)", onde o neurônio artificial é excitado ou não baseado em um treshold (limiar) pré definido.
Nesse caso, se o valor do neurônio for maior que 0 ele retorna o 1, excitando a célula, caso contrário, retorna 0

In [2]:
def funcao_ativacao(soma):
    if soma > 0:
        return 1
    return 0

## Função de atualização de peso
A atualização de pesos no perceptron é definida pela seguinte formula:


Com isso, conseguimos ajustar os pesos seguindo um taxa de aprendizado como o tamanho do nosso "passo" além de levar em consideração a grandeza da entrada e o quanto o valor errou.

In [3]:
def atualizar_peso(entrada, peso, erro, tx_aprendizado = 0.2):
    novo_peso = peso + (tx_aprendizado * entrada * erro)
    print('peso atualizado', novo_peso)
    return novo_peso

## Implementação do perceptron
A função de treinamento recebe o número de épocas que iremos executar nosso perceptron, onde cada época representa a passagem de todo o dataset na nossa rede. A estratégia seguida a princípio para atualização dos pesos é por registro, ou seja, para cada registro errado nós fazemos os ajustes nos pesos. Geralmente no perceptron multicamadas essa atualização é feita por épocas ou batches.

In [ ]:
def treinar(epocas):
    execucoes = 0
    while execucoes < epocas:
        precisao = 100
        iteracao = 0

        np.random.shuffle(previsores.values) # embaralhar os valores dos previsores, por que sem isso, podemos ter sempre uma ordem fixa de ajuste de pesos, prejudicando a rede

        for i in previsores.values:
            entradas = i   
            soma = somatoria(entradas, pesos)
        
            ativacao = funcao_ativacao(soma)
        
            erro = funcao_custo(classe[iteracao], ativacao) # baseado no meu resultado previsto, dado na última função de ativação.
        
            if erro > 0:
                precisao -= 100 / len(previsores) 
                print('Precisão: ', precisao)
                count = 0
                    
                for i in entradas:
                    novo_peso = atualizar_peso(i, pesos[count], erro)
                    pesos[count] = novo_peso
                    count += 1
            
            iteracao += 1
        
        execucoes += 1
    print('Precisão final: ', precisao)

In [ ]:
treinar(20)